In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By


In [2]:
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
from io import StringIO
from csv import writer 
import datetime as dt

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - There is no [win32] chromedriver for browser 104.0.5112 in cache
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/104.0.5112.79/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Panflete\.wdm\drivers\chromedriver\win32\104.0.5112.79]
C:\Users\Panflete\AppData\Local\Temp\ipykernel_36808\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
url = "http://old.emmsa.com.pe/emmsa_spv/rpEstadistica/rptVolPreciosDiarios.php"
driver.get(url)

In [5]:
frame = driver.find_element(by=By.XPATH,value="/html/body/div/div/section/div/div/div/section/div/div/div/iframe")

In [6]:
driver.switch_to.frame(frame)

In [7]:
dateInput = driver.find_element(By.XPATH,"/html/body/div[1]/div[2]/div[2]/div/table/tbody/tr[1]/td[1]/div/div[2]/input")

In [14]:
checkCebolla = driver.find_element(By.ID,14)
checkCebolla.click()
checkCebolla = driver.find_element(By.ID,38)
checkCebolla.click()

In [15]:
consultarButton = driver.find_element(By.CLASS_NAME,"button")
consultarButton.click()

In [17]:
output = StringIO()
csv_writer = writer(output)

csv_writer.writerow(["producto","year","month","day", "precio_min", "precio_max", "precio_mean"])

endDate = dt.date(2022,8,30)
currentDate = dt.date(2022,5,1)
while endDate >= currentDate:
	dateInput.clear()
	dateInput.send_keys("value",currentDate.strftime("%d/%m/%Y"))

	consultarButton = driver.find_element(By.CLASS_NAME,"button")
	consultarButton.click()
	table = None
	while(table == None):
		table = bs(driver.page_source).find("table", {"class":"timecard"})

	body = table.find("tbody")
	for row in body.find_all("tr"):
		new_row = [str(col).replace("<td>","").replace("</td>","") for col in row.findAll("td")[1:]]
		new_row.insert(1,str(currentDate.day))
		new_row.insert(1,str(currentDate.month))
		new_row.insert(1,str(currentDate.year))
		csv_writer.writerow(new_row)
	currentDate -= dt.timedelta(days=1)

In [76]:
output.seek(0)
data = pd.read_csv(output)

In [77]:
data.to_csv("./data2.csv",index=False)